In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Step 0: Unzip data file and read data
## Unzip file
DATA_PATH = Path().absolute().parent / "data"

# Read parquets
DATAFRAMES = {
    folder: {
        data_file: pd.read_parquet(DATA_PATH / folder / data_file)
        for data_file in os.listdir(DATA_PATH / folder)
        if Path(data_file).suffix == ".pq"
    }
    for folder in os.listdir(DATA_PATH)
}

# Now you can access each DataFrame using its folder & file name as the key in the 'dataframes' dictionary
# For example:
# df_pc_price_eu = DATAFRAMES["pc_price"]["pc_price_eu.pq"]

# PC Price

In [ ]:
df_pc_price_eu = DATAFRAMES["pc_price"]["pc_price_eu.pq"]
df_pc_price_eu.head()

## Comparison of Polycarbonate Prices Across Suppliers

In [ ]:
df_best_pc_price_eu = (
    df_pc_price_eu.set_index("date")[df_pc_price_eu.columns[df_pc_price_eu.columns.str.startswith("eu_supplier")]]
    .min(axis=1)
    .reset_index(drop=False)
    .rename(columns={0: "price_pc_eur_per_kg"})
)

# Create figure for the best price
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_best_pc_price_eu["date"],
        y=df_best_pc_price_eu["price_pc_eur_per_kg"],
        mode="lines+markers",
        name="Best Price",
        marker={"size": 5, "color": "red"},
        line={"color": "red"},
    )
)

# Add traces for each EU supplier
for col in df_pc_price_eu.columns:
    if col.startswith("eu_supplier"):
        fig.add_trace(
            go.Scatter(
                x=df_pc_price_eu["date"],
                y=df_pc_price_eu[col],
                mode="lines",
                name=col,
                line={"dash": "dash", "width": 2},
                opacity=0.5,
            )
        )

fig.update_layout(
    title={
        "text": "PolyCarbonate prices in EUR/Kg accross SE suppliers, against time",
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
        "font": {"size": 20},
    },
    xaxis_title="Date",
    yaxis_title="Polycarbonate price (EUR/Kg)",
    width=1150,
    height=600,
    template="plotly",
)

fig.show("svg")

- **General Price Trend**: Prices from different suppliers vary but follow a similar trend, likely reflecting broader market conditions. The range in pricing among suppliers suggests potential opportunities for cost optimization by selecting or negotiating with certain suppliers.
Best Price Identification: By identifying the lowest price among suppliers at each point in time, SE can target the most competitive suppliers for procurement. Based on this analysis, "eu_supplier_1 - pc crystal" and "eu_supplier_3 - pc crystal" emerge as cost-effective choices for SE, allowing for informed purchasing and improved negotiation strategies.

## Distribution of the Best Polycarbonate Price (EUR/Kg)

In [ ]:
# Distribution of the best price
import plotly.figure_factory as ff

group_labels = ["Distribution of Best PC Price (EUR/Kg)"]

fig = ff.create_distplot(
    hist_data=[df_best_pc_price_eu["price_pc_eur_per_kg"].to_numpy()],
    group_labels=group_labels,
    bin_size=0.2,
    show_rug=False,
)

fig.update(layout_title_text="Distribution of Best PC Price (EUR/Kg)")
fig.update_layout(showlegend=False)
fig.show("svg")

## Polycarbonate Price vs. Electricity Price in France

In [ ]:
# PC Price vs. Electricity price
df_electricity_price = DATAFRAMES["electricity_price"]["electricity_price_history_per_country.pq"]
df_electricity_price = df_electricity_price.loc[
    df_electricity_price.country.str.startswith("FR"), ["date", "price - EUR/MWh (avg)"]
]
df_electricity_price_pc_price = (
    df_electricity_price.assign(date=lambda df: pd.to_datetime(df["date"], format="%Y/%m").dt.strftime("%b-%Y"))
    .merge(df_best_pc_price_eu, how="right", on="date")
    .dropna()
    .rename(columns={"price - EUR/MWh (avg)": "electricity_price_eur_per_mwh"})
)

fig = go.Figure()
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Scatter(
        x=df_electricity_price_pc_price["date"],
        y=df_electricity_price_pc_price["price_pc_eur_per_kg"],
        mode="lines+markers",
        name="Best PC Price (EUR/Kg)",
        marker={"size": 5, "color": "red"},
        line={"color": "red"},
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=df_electricity_price_pc_price["date"],
        y=df_electricity_price_pc_price["electricity_price_eur_per_mwh"],
        mode="lines+markers",
        name="Electricity Price (EUR/MWh)",
        marker={"size": 5, "color": "blue"},
        line={"color": "blue"},
    ),
    row=2,
    col=1,
)

fig.update_layout(
    title={
        "text": "Best PC price vs. Electricity Price",
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
        "font": {"size": 20},
    },
    xaxis_title="Date",
    yaxis_title="Price",
    width=1150,
    height=1000,
    template="plotly",
)

fig.show("svg")

## Analysis of Spreads between Raw Materials and Polycarbonate

In [ ]:
# Strip any leading/trailing whitespace from column names
df_pc_price_eu.columns = df_pc_price_eu.columns.str.strip()

# Extract the spreads and polycarbonate price columns
df_spreads = df_pc_price_eu[["date", "spread oil / benzene", "spread benzene / bpa", "spread bpa / pc", "pc pie"]]

# Convert date to datetime format for easier visualization
df_spreads["date"] = pd.to_datetime(df_spreads["date"], format="%b-%Y")

# Create a figure with multiple subplots for each spread
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Plot each spread separately
fig.add_trace(
    go.Scatter(x=df_spreads["date"], y=df_spreads["spread oil / benzene"], mode="lines", name="Spread Oil / Benzene"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df_spreads["date"], y=df_spreads["spread benzene / bpa"], mode="lines", name="Spread Benzene / BPA"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df_spreads["date"], y=df_spreads["spread bpa / pc"], mode="lines", name="Spread BPA / PC"),
    row=3, col=1
)

# Set the layout of the figure
fig.update_layout(
    title="Spreads between Raw Materials and Polycarbonate",
    xaxis_title="Date",
    yaxis_title="Spread",
    height=900,
    template="plotly",
)

fig.show("svg")


## Seasonal decomposition of polycarbonate prices

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# If "date" is the index, reset it to make it a column
df_best_pc_price_eu = df_best_pc_price_eu.reset_index()

# Convert "date" column to datetime if it isn't already
df_best_pc_price_eu["date"] = pd.to_datetime(df_best_pc_price_eu["date"], format="%b-%Y")

# Set "date" as the index for time series analysis
df_best_pc_price_eu.set_index("date", inplace=True)

# Perform seasonal decomposition
result = seasonal_decompose(df_best_pc_price_eu["price_pc_eur_per_kg"], model="multiplicative", period=12)

# Plot the decomposition
result.plot()
plt.suptitle("Seasonal Decomposition of Best Polycarbonate Price", y=1.05)
plt.show()


- **Trend**: The trend component shows a rise in polycarbonate prices starting around 2021, peaking in 2022 before slightly declining. This trend may reflect supply and demand shocks linked to macroeconomic events, like the COVID-19 pandemic or supply chain disruptions. Further analysis of shutdowns or capacity losses for phenol and acetone, which are essential components in polycarbonate production, could help explain these fluctuations.

- **Seasonal**: The seasonal component shows a recurring but low-amplitude variation, indicating moderate seasonal influences on demand or production. The stability in seasonality suggests that polycarbonate demand is less affected by strong seasonal cycles, unlike more season-sensitive products.

- **Residual**: The residuals are relatively low and lack marked trends, indicating that the decomposition model effectively captures the main seasonal and trend components. This also suggests limited impact from unexpected market fluctuations, reinforcing the polycarbonate market's stability.

## Correlation matrix between raw material spreads and polycarbonate price

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate correlation matrix
correlation_matrix = df_spreads[["spread oil / benzene", "spread benzene / bpa", "spread bpa / pc", "pc pie"]].corr()

# Plot the correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Matrix between Raw Material Spreads and Polycarbonate Price")
plt.show()


- **Spread oil / benzene vs. polycarbonate price (0.36)**: A moderate positive correlation between the oil/benzene spread and polycarbonate price suggests that as the price gap between oil and benzene increases, polycarbonate prices tend to rise as well. Oil is a key input for benzene production, which is a building block in the polycarbonate supply chain. SE could leverage this link to predict polycarbonate price shifts based on oil market trends.

- **Spread benzene / BPA vs. polycarbonate price (-0.67)**: The strong negative correlation implies that when the cost of converting benzene to BPA decreases, polycarbonate prices tend to rise. This could suggest an inverse market response to the chemical conversion costs, potentially due to increased BPA production, which drives up polycarbonate demand and price. SE could further explore this relationship to enhance forecasting.

- **Spread BPA / polycarbonate vs. polycarbonate price (-0.23)**: This weaker correlation suggests that fluctuations in the price gap between BPA and polycarbonate have limited impact on polycarbonate pricing. Other factors may play a more significant role in determining final polycarbonate prices, such as demand patterns or alternative cost drivers.

## Volatility analysis of polycarbonate prices per supplier

In [ ]:
# Select columns related to suppliers in df_pc_price_eu
df_suppliers = df_pc_price_eu.set_index("date").filter(like="eu_supplier")

# Ensure 'date' is properly formatted
df_suppliers.index = pd.to_datetime(df_suppliers.index, format="%b-%Y")

# Calculate rolling standard deviation (volatility) for each supplier over a 3-month window
rolling_std = df_suppliers.rolling(window=3).std()

# Plot volatility of each supplier's prices
fig = go.Figure()

for supplier in rolling_std.columns:
    fig.add_trace(
        go.Scatter(x=rolling_std.index, y=rolling_std[supplier], mode="lines", name=f"Volatility - {supplier}")
    )

fig.update_layout(
    title="Rolling 3-Month Volatility in Polycarbonate Prices per Supplier",
    xaxis_title="Date",
    yaxis_title="Price Volatility (EUR/Kg)",
    template="plotly",
    width=1150,
    height=600,
)

fig.show("svg")


- **Volatility by Supplier**: This chart displays the average 3-month price volatility for each European polycarbonate supplier. Higher volatility indicates greater pricing uncertainty, which could stem from production challenges or varying pricing strategies among suppliers.

- **Supplier Comparison**: "eu_supplier_1 - pc crystal" consistently shows the lowest volatility, indicating price stability, while "eu_supplier_2 - pc crystal" and "eu_supplier_5 - pc crystal" have higher volatility, reflecting less predictable pricing. SE could prioritize suppliers like "eu_supplier_1 - pc crystal" to reduce exposure to price fluctuations, providing more reliable cost management.

## Histogram of monthly price changes for Polycarbonate

In [ ]:
# Calculate monthly price changes
df_price_change = df_best_pc_price_eu["price_pc_eur_per_kg"].pct_change() * 100  # Percentage change

# Plot histogram of monthly price changes
plt.figure(figsize=(10, 6))
plt.hist(df_price_change.dropna(), bins=20, edgecolor="k", alpha=0.7)
plt.title("Distribution of Monthly Price Changes for Best Polycarbonate Price")
plt.xlabel("Monthly Change (%)")
plt.ylabel("Frequency")
plt.show()


- **General Observation**: The distribution is tightly concentrated around zero, indicating that most months see little to no change in polycarbonate prices. This stability is characteristic of a relatively steady market with infrequent large fluctuations.

- **Extreme Fluctuations**: A few isolated points show significant positive and negative changes, suggesting occasional volatility. These extremes may coincide with specific market shocks or events. For instance, sudden price increases could result from supply chain disruptions, while drops could reflect decreased demand. Further investigation into these points and their correlation with macroeconomic events or shutdown data could clarify their causes.

# Commodity price

In [ ]:
# Load the Europe sheet
df_raw = pd.read_excel(DATA_PATH / "commodity_price" / "Commodity_Price_Watch_Global Tables_January_2024_Raul_Granados (2).xlsx", sheet_name="Europe", header=None)

# Identify the row index where the dates are located
date_row_idx = 8  # Adjust this index based on the actual row with dates in your file

# Extract dates as column headers
dates = df_raw.iloc[date_row_idx, 2:].tolist()

# Dictionary to store section data
sections = {
    "Energy Prices": "Energy Prices",
    "Chemical Market Analytics": "Chemical Market Analytics",
    "Steel Spot Prices": "Steel spot prices",
    "Nonferrous Metals, LME Prices": "Nonferrous metals, LME prices",
    "Stainless Steel and Ferro-Alloys Prices": "Stainless steel and ferro-alloys prices"
}

section_dataframes = {}

# Loop through each section and extract data
for section_name, keyword in sections.items():
    # Find the start of the section
    start_row = df_raw[df_raw[0].astype(str).str.contains(keyword, case=False, na=False)].index[0] + 1

    # Determine the end of the section (next section start or empty row)
    end_row = start_row
    while end_row < len(df_raw) and not pd.isna(df_raw.iloc[end_row, 0]):
        end_row += 1

    # Extract and clean the section
    section_data = df_raw.iloc[start_row:end_row].reset_index(drop=True)
    
    # Set the first row as labels for product names, the second as units, and the extracted dates as column headers
    section_data.columns = ["Product", "Unit"] + dates[:len(section_data.columns) - 2]
    section_data = section_data.drop(0).reset_index(drop=True)  # Remove the row with units as data

    # Store the cleaned section in the dictionary
    section_dataframes[section_name] = section_data

# Display the cleaned data for each section to verify structure
for section, df in section_dataframes.items():
    print(f"Data for section: {section}")
    print(df.head())


## Energy Prices Over Time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

def read_commodity_data_adjusted(file_path, sheet_name='Europe'):
    """
    Reads Excel file with commodity price data, handling variable column numbers.
    """
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Identify the date row (row 9 in file, index 8)
    date_row_idx = 8
    dates = df_raw.iloc[date_row_idx, 2:].dropna().tolist()
    
    # Filter valid quarterly dates
    dates = [date for date in dates if isinstance(date, str) and date.startswith('Q')]
    
    sections = {
        "Energy Prices": "Energy Prices",
        "Chemical Market Analytics": "Chemical Market Analytics",
        "Steel Spot Prices": "Steel spot prices",
        "Nonferrous Metals, LME Prices": "Nonferrous metals, LME prices",
        "Stainless Steel and Ferro-Alloys Prices": "Stainless steel and ferro-alloys prices"
    }
    
    section_dataframes = {}
    current_section = None
    section_start = None
    
    for idx, row in df_raw.iterrows():
        first_cell = str(row.iloc[0]) if not pd.isna(row.iloc[0]) else ""
        
        # Check if this row starts a new section
        for section_name, keyword in sections.items():
            if keyword.lower() in first_cell.lower():
                # Process previous section if it exists
                if current_section and section_start is not None:
                    section_data = process_section(df_raw, section_start, idx, dates)
                    if section_data is not None:
                        section_dataframes[current_section] = section_data
                
                current_section = section_name
                section_start = idx + 1
                break
        
        # Process section if we hit an empty row
        if pd.isna(first_cell) and current_section and section_start is not None:
            section_data = process_section(df_raw, section_start, idx, dates)
            if section_data is not None:
                section_dataframes[current_section] = section_data
            current_section = None
            section_start = None
    
    # Process final section if it exists
    if current_section and section_start is not None:
        section_data = process_section(df_raw, section_start, len(df_raw), dates)
        if section_data is not None:
            section_dataframes[current_section] = section_data
    
    return section_dataframes

def process_section(df_raw, start_idx, end_idx, dates):
    """
    Process a section of the data, handling column count mismatches.
    """
    section_data = df_raw.iloc[start_idx:end_idx].copy()
    
    # Drop completely empty columns and rows
    section_data = section_data.dropna(axis=1, how='all')
    section_data = section_data.dropna(axis=0, how='all')
    
    if section_data.empty:
        return None
    
    # Get actual number of columns in the data
    num_columns = section_data.shape[1]
    
    # Create column names based on actual data
    if num_columns >= 2:  # Ensure we have at least Product and Unit columns
        col_names = ["Product", "Unit"]
        remaining_cols = num_columns - 2
        col_names.extend(dates[:remaining_cols])
        
        # Ensure we have the right number of column names
        while len(col_names) < num_columns:
            col_names.append(f"Extra_{len(col_names)}")
        
        section_data.columns = col_names[:num_columns]
        return section_data.iloc[1:].reset_index(drop=True)
    
    return None

def clean_and_convert_dates(df):
    """
    Clean and convert date columns to datetime format.
    """
    date_cols = []
    for col in df.columns[2:]:  # Skip Product and Unit columns
        if isinstance(col, str) and col.startswith('Q'):
            try:
                quarter = int(col[1])
                year = int(col.split()[1])
                month = (quarter - 1) * 3 + 1
                date = pd.to_datetime(f'{year}-{month:02d}-01')
                date_cols.append((col, date))
            except:
                continue
    return sorted(date_cols, key=lambda x: x[1])

def plot_commodity_trends(section_df, section_name):
    """
    Create a plot of commodity price trends over time.
    """
    if section_df is None or section_df.empty:
        print(f"No data available for section {section_name}")
        return
    
    df = section_df.copy()
    valid_date_cols = clean_and_convert_dates(df)
    
    if not valid_date_cols:
        print(f"No valid date columns found for {section_name}")
        return
    
    # Prepare data for plotting
    plot_df = df[['Product'] + [col[0] for col in valid_date_cols]].set_index('Product')
    plot_df = plot_df.apply(pd.to_numeric, errors='coerce')
    
    # Create plot
    plt.figure(figsize=(15, 8))
    for product in plot_df.index:
        plt.plot(plot_df.columns, plot_df.loc[product], marker='o', label=product)
    
    plt.title(f"{section_name} - Price Trends Over Time")
    plt.xlabel("Date")
    plt.ylabel("Price (in respective units)")
    plt.legend(title="Products", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Usage example
def analyze_commodity_prices(data_path):
    """
    Analyze and plot commodity prices from the Excel file.
    """
    try:
        # Define the path to the Excel file
        file_path = data_path / "commodity_price" / "Commodity_Price_Watch_Global Tables_January_2024_Raul_Granados (2).xlsx"
        
        # Read and process the data
        data = read_commodity_data_adjusted(file_path)
        
        # Plot the data for each section
        for section_name, df in data.items():
            print(f"\nAnalyzing {section_name}...")
            plot_commodity_trends(df, section_name)
    except Exception as e:
        print(f"Error analyzing commodity prices: {str(e)}")

# Define the data path
DATA_PATH = Path().absolute().parent / "data"

# Analyze the commodity prices
analyze_commodity_prices(DATA_PATH)